# Face recognition learning pipeline for VGGface dataset

## Подключение библиотек

In [582]:
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np
import glob
import cv2
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import time
from torch.autograd import Variable
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.onnx
from torchinfo import summary
from typing import Tuple
import torchvision.models as models

## Используемые функции

In [5]:
# Добавить функцию по скачиванию изображений VGGface, сделать многопоточное скачивание

In [6]:
# Добавить функцию по выделению лиц из скачанных изображений
# можно расширить bbox, чтобы больше лица влезло в кадр

In [ ]:
# сделать обучение на vggface2
# перебрать гиперпараметры
# сделать learning rate annealing
# разморозить больше слоев в сети и обучить

In [ ]:
# разобраться как работают num_workers у Dataliader

In [395]:
def get_mean_std(research_data_loader: DataLoader) -> Tuple[tuple]:
    '''Функция получает на вход объект класса DataLoader для 
    вычисления mean и std для всего датасета поканально
    
    Входные параметры:
    research_data_loader: DataLoader - объект для загрузки изображений лиц
    Возвращаемые значения:
    (mean, std): Tuple[tuple] - кортеж, содержащий кортежи со значениями mean и std поканально
    ((mean_r, mean_g, mean_b), (std_r, std_g, std_b))'''
    nimages = 0
    mean = 0.
    std = 0.
    for batch in research_data_loader:
        # Приводим тензор из [B, C, W, H] к [B, C, W * H]
        batch = batch.view(batch.size(0), batch.size(1), -1)
        # Суммируем общее количество иобработанных изображений
        nimages += batch.size(0)
        # Вычисляем mean и std 
        mean += batch.mean(2).sum(0) 
        std += batch.std(2).sum(0)

    mean /= nimages
    std /= nimages
    return tuple(mean.cpu().numpy()), tuple(std.cpu().numpy())

In [396]:
def get_data_csv(faces_directory_path: str = None) -> pd.DataFrame:
    '''Функция получает на вход путь к директории с лицами
    и генерирует датафрейм, содержащий полные имена изображений 
  
    Входные параметры:
    faces_directory_path: str - путь к директории с лицами,
    Возвращаемые значения:
    pd.DataFrame: data - dataframe, содержащий полные имена изображений'''

    if not os.path.isdir(faces_directory_path):
        raise OSError('Directory is not exist')

    data = {}
    data['face_path'] = []
    data['face_path'] = list(glob.glob(faces_directory_path + "/*/*"))
    data = pd.DataFrame(data)
    data['person_name'] = data['face_path'].apply(lambda x: x.split('/')[-2])
    #data = data.sort_values(by='person_name')

    return data

In [419]:
def get_train_valid_test(source_df: pd.DataFrame, separate_feature: str = None, 
                         valid_size: float = 0.2, test_size: float = 0.2) -> pd.DataFrame:
    '''Функция разделяет source_df на три части с коэффициентами valid_size и test_size
    по уникальным значениям separate_feature так, чтобы в новых датафреймах
    не было строк с одинаковыми значениями из separate_feature

    Входные параметры:
    source_df: pd.DataFrame - датафрейм для разделения на train, valid и test
    separate_feature: str - поле, по которому датафрейм будет разделен
    valid_size: float - коэффициент разделения для valid
    test_size: float - коэффициент разделения для test
    Возвращаемые значения:
    pd.DataFrame: data_train - датафрейм для тренировки
    pd.DataFrame: data_valid - датафрейм для валидации
    pd.DataFrame: data_test - датафрейм для тестирования'''
  
    if (separate_feature != None) & (separate_feature in source_df.columns):
        train_faces, test_faces = train_test_split(source_df[separate_feature].unique(), 
                                                   test_size=(valid_size + test_size), random_state=42)
        valid_faces, test_faces = train_test_split(test_faces, 
                                                   test_size=(test_size/(valid_size + test_size)), random_state=42)
        
        data_train = source_df[np.isin(source_df[separate_feature].values, train_faces)]
        data_valid = source_df[np.isin(source_df[separate_feature].values, valid_faces)]
        data_test = source_df[np.isin(source_df[separate_feature].values, test_faces)]
        
        assert source_df.shape[0] == (data_train.shape[0] + data_valid.shape[0] + data_test.shape[0])
        assert np.isin(data_train[separate_feature].values, data_valid[separate_feature].values).sum() == 0
        assert np.isin(data_train[separate_feature].values, data_test[separate_feature].values).sum() == 0
        assert np.isin(data_valid[separate_feature].values, data_test[separate_feature].values).sum() == 0
        
    else:
        data_train, data_test = train_test_split(source_df, test_size=(valid_size + test_size))
        data_valid, data_test = train_test_split(data_test, test_size=(test_size/(valid_size + test_size)))
        

    return (data_train, data_valid, data_test)

## Используемые классы

In [398]:
class ResearchDataset(Dataset):
    '''Класс для создания датасета для вычисления среднего и дисперсии по каналам изображений лиц'''
    def __init__(self, data_info: pd.DataFrame, device: str):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами изображений и масок
        device: str - имя устройства, на котором будут обрабатываться данные
        transform: object - список трансформации, которым будут подвергнуты изображения и маски
        Возвращаемые значения:
        объект класса CustomDatasetForTrain'''
        # Подаем подготовленный датафрейм
        self.data_info = data_info
        # Количество изображений
        self.data_len = len(self.data_info.index)
        self.device = device

    def __getitem__(self, index: int):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        image_tensor: torch.Tensor - тензорное представление изображения лица'''
        
        image_numpy = cv2.imread(self.data_info['face_path'][index])
        image_numpy = cv2.cvtColor(image_numpy, cv2.COLOR_BGR2RGB).astype('float')/255.0
        image_tensor = torch.from_numpy(image_numpy)
        image_tensor = image_tensor.to(self.device).float()
        image_tensor = image_tensor.permute(2, 0, 1)

        return image_tensor

    def __len__(self):
        return self.data_len

In [414]:
class CustomDataset(Dataset):
    '''Класс для создания тренировочных и валидационных датасетов'''
    def __init__(self, data_info: pd.DataFrame, device: str, transform: object):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами изображений и масок
        device: str - имя устройства, на котором будут обрабатываться данные
        transform: object - список трансформации, которым будут подвергнуты изображения и маски
        Возвращаемые значения:
        объект класса CustomDatasetForTrain'''
        # Подаем подготовленный датафрейм
        self.data_info = data_info
        # 
        self.face_path_arr = self.data_info.iloc[:,0]
        # 
        self.person_name_arr = self.data_info.iloc[:,1]
        # Количество изображений
        self.data_len = len(self.data_info.index)
        # Устройство, на котором будут находиться выходные тензоры
        self.device = device
        # Сохраняем преобразования данных
        self.transform = transform

    def __getitem__(self, index: int):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        img: torch.Tensor - тензорное представление изображения лица'''
        
        anchor_image = cv2.imread(self.data_info['face_path'][index])
        anchor_person_name = self.data_info['person_name'][index]
        
        positive_indices = self.data_info[self.data_info['person_name'] == anchor_person_name].index.values
        negative_indices = self.data_info[self.data_info['person_name'] != anchor_person_name].index.values
        
        while True:
            positive_index = np.random.choice(positive_indices)
            if positive_index != index:
                break
        
        negative_index = np.random.choice(negative_indices)
        
        
        positive_image = cv2.imread(self.face_path_arr[positive_index])
        negative_image = cv2.imread(self.face_path_arr[negative_index])
        
        anchor_image = cv2.cvtColor(anchor_image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        positive_image = cv2.cvtColor(positive_image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        negative_image = cv2.cvtColor(negative_image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        
        tr_anchor_image = self.transform(image=anchor_image)['image'].to(self.device).float()
        tr_positive_image = self.transform(image=positive_image)['image'].to(self.device).float()
        tr_negative_image = self.transform(image=negative_image)['image'].to(self.device).float()

        return tr_anchor_image, tr_positive_image, tr_negative_image

    def __len__(self):
        return self.data_len

In [473]:
class TripletLoss(nn.Module):
    '''Класс для вычисления Triplet loss для набора изображенй в формате torch.Tensor'''
    def __init__(self, margin: float):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor_embs: torch.Tensor, positive_embs: torch.Tensor, negative_embs: torch.Tensor) -> float:
        '''Входные параметры:
        anchor_embs: torch.Tensor - тензор предсказанных эмбэддингов для якорей
        positive_embs: torch.Tensor - тензор предсказанных эмбэддингов для позитивных примеров
        negative_embs: torch.Tensor - тензор предсказанных эмбэддингов для негативных примеров
        все тензоры имеют размерность B х E, где B - размер батча, E - размер вектора эмбэддинга
        
        Возвращаемые значения:
        score: float - значение Triplet loss для наборов предсказанных эмбэддингов'''
        
        anchor_positive_distances = torch.sum((anchor_embs - positive_embs)**2, dim=1)
        anchor_negative_distances = torch.sum((anchor_embs - negative_embs)**2, dim=1)
        
        print(anchor_positive_distances)
        
        score = torch.max(0, anchor_positive_distances - anchor_negative_distances + self.margin)
        
        return score

In [561]:
class NeuralNetwork(nn.Module):
    '''Класс для работы с нейронной сетью для распознавания лиц на основе VGGface'''
    def __init__(self, model: object):
        '''Конструктор класса
        Входные параметры:
        model: nn.Module - последовательность слоев или модель, через которую будут проходить данные
        Возвращаемые значения: 
        объект класса NeuralNetwork'''
        super(NeuralNetwork, self).__init__()
        self.model = model

    def forward(self, input_data):
        '''Функция прямого прохода через объкт класса
        Входные параметры:
        input_data: torch.Tensor - тензорное представление изображения
        Возвращаемые значения: 
        input_data: torch.Tensor - тензорное представление маски изображения'''
        output_data = self.model(input_data)
        return output_data
    
    
    def fit(self, criterion: object, metric: object, optimizer: object, 
                  train_data_loader: DataLoader, valid_data_loader: DataLoader=None, 
                  epochs: int=1, verbose: int=50):
        '''Метод для обучения модели
        Входные параметры:
        criterion: object - объект для вычисления loss
        metric: object - объект для вычисления метрики качества
        optimizer: object - оптимизатор
        train_data_loader: DataLoader - загрузчик данных для обучения
        valid_data_loader: DataLoader - загрузчик данных для валидации
        epochs: int - количество эпох обучения
        verbose: int - вывод информации через каждые verbose итераций
        
        Возвращаемые значения:
        result: dict - словарь со значениями loss при тренировке, валидации и метрики при валидации 
        для каждой эпохи'''
        
        self.optimizer = optimizer
        epoch_train_losses = []
        epoch_valid_losses = []
        epoch_valid_metrics = []
        result = {}
        
        for epoch in range(epochs):
            self.model.train()
            time1 = time.time()
            running_loss = 0.0
            train_losses = []
            for batch_idx, data in enumerate(train_data_loader):
                # data - список из 3-х тезоров с размерностями [B, C, W, H]. Сделаем конкатенацию
                # в один тензор с размерностью [3*B, C, W, H] для одновременного прогона через сеть
                data = torch.cat(data, dim=0)
                data = Variable(data)       

                optimizer.zero_grad()
                outputs = self.model(data)
                # После прогона данных, разделяем результат на три соответствующих тензора
                batch_size = int(outputs.shape[0]/3)
                anchor_embs = outputs[:batch_size, :]
                positive_embs = outputs[batch_size : 2*batch_size, :]
                negative_embs = outputs[2*batch_size :, :]
                
                loss = criterion(anchor_embs, positive_embs, negative_embs)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                train_losses.append(loss.item())
                if (batch_idx+1) % verbose == 0:
                    print(f'Train Epoch: {epoch+1}, Loss: {(running_loss/verbose):.6f}')
                    time2 = time.time()
                    print(f'Spended time for {verbose} batches ({int((verbose*data.shape[0])/3)} triplets', end="") 
                    print(f' or {int(verbose*data.shape[0])} images) : {(time2-time1):.6f} sec')
                    
                    time1 = time.time()
                    running_loss = 0.0

            train_loss = np.mean(train_losses)        
            
            if valid_data_loader != None:
                valid_result = self.valid(criterion, metric, valid_data_loader)
                valid_loss = valid_result['valid_loss']
                valid_metric = valid_result['valid_metric']
            
                print('='*80)
                print(f'Epoch {epoch+1}, train loss: {(train_loss):.6f}, valid_loss: {(valid_loss):.6f}, ', end="")
                print(f'valid_metric: {(valid_metric):.6f}')
                print('='*80)
            else:
                print('='*80)
                print(f'Epoch {epoch+1}, train loss: {(train_loss):.6f}')
                print('='*80)
                valid_loss = None
                valid_metric = None
            
            epoch_train_losses.append(train_loss)
            epoch_valid_losses.append(valid_loss)
            epoch_valid_metrics.append(valid_metric)
        
        result['epoch_train_losses'] = epoch_train_losses
        result['epoch_valid_losses'] = epoch_valid_losses
        result['epoch_valid_metrics'] = epoch_valid_metrics
        
        return result
    
    def valid(self, criterion: object, metric: object, valid_data_loader: DataLoader):
        '''Метод для валидации модели
        Входные параметры:
        criterion: object - объект для вычисления loss
        metric: object - объект для вычисления метрики качества
        valid_data_loader: DataLoader - загрузчик данных для валидации
        
        Возвращаемые значения:
        result: dict - словарь со значениями loss и метрики при валидации'''
        self.model.eval()
        valid_metrics = []
        valid_losses = []
        result = {}
        for batch_idx, data in enumerate(valid_data_loader):
            data = torch.cat(data, dim=0)
            data = Variable(data)
            
            outputs = self.model(data)
                    
            batch_size = int(outputs.shape[0]/3)
            anchor_embs = outputs[:batch_size, :]
            positive_embs = outputs[batch_size : 2*batch_size, :]
            negative_embs = outputs[2*batch_size :, :]
            
            loss = criterion(anchor_embs, positive_embs, negative_embs)
            valid_losses.append(loss.item())
                    
            metric_value = metric(anchor_embs, positive_embs, negative_embs)
            valid_metrics.append(metric_value.item())
                
        valid_loss    = np.mean(valid_losses)
        valid_metric  = np.mean(valid_metrics)
        result['valid_loss'] = valid_loss
        result['valid_metric'] = valid_metric
        return result

    
    
    def save(self, path_to_save: str='./model.pth'):
        '''Метод сохранения весов модели
        Входные параметры:
        path_to_save: str - директория для сохранения состояния модели'''
        torch.save(self.model.state_dict(), path_to_save)
    
    def trace_save(self, path_to_save: str='./model.pth'):
        '''Метод сохранения модели через torchscript
        Входные параметры:
        path_to_save: str - директория для сохранения модели'''
        example_forward_input = torch.rand(1, 3, 512, 512).to('cpu')
        if next(self.model.parameters()).is_cuda:
            example_forward_input= example_forward_input.to('cuda:0')
            
        traced_model = torch.jit.trace((self.model).eval(), example_forward_input)
        torch.jit.save(traced_model, path_to_save)
    
    def onnx_save(self, path_to_save: str='./carvana_model.onnx'):
        '''Метод сохранения модели в формате ONNX
        Входные параметры:
        path_to_save: str - директория для сохранения модели'''
        example_forward_input = torch.randn(1, 3, 1024, 1024, requires_grad=True).to('cpu')
        if next(self.model.parameters()).is_cuda:
            example_forward_input= example_forward_input.to('cuda:0')

        torch.onnx.export(self.model,
                          example_forward_input,
                          path_to_save,
                          export_params=True,
                          opset_version=10,
                          do_constant_folding=True,
                          input_names = ['input'],
                          output_names = ['output'],
                          dynamic_axes={'input' : {0 : 'batch_size'},    # Модель будет работать с произвольным
                                        'output' : {0 : 'batch_size'}})  # размером батча
    
    def load(self, path_to_model: str='./model.pth'):
        '''Метод загрузки весов модели
        Входные параметры:
        path_to_model: str - директория с сохраненными весами модели'''
        self.model.load_state_dict(torch.load(path_to_model))

In [562]:
# Собственная модель с нуля
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.alobal_average = nn.AvgPool2d(5)
        
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.4)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        
        x = self.alobal_average(x)
        
        x = x.view(-1, 512)
        x = F.relu(self.dropout1(self.fc1(x)))
        x = self.fc2(x)
        return x

In [607]:
class Identity(nn.Module):
    '''Класс необходим для реализации transfer learning.
    Если создать объект этого класса и присвоить его последнему слою классификатора
    обученной нейросети, то классификатор перезапишется, сигнал будет проходить без изменений, 
    и в дальнейшем можно будет добавлять пользовательские слои'''
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [624]:
# Постороение модели на основе mobilenet_v3_small
class MobileImagenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.mobilenet_v3_small(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        self.backbone.classifier[-1] = Identity()

        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout1 = nn.Dropout(p=0.3)

    def get_layers_names(self):
         return dict(self.backbone.named_modules())

    def forward(self, x):
        x = self.backbone(x)
        x = F.relu(self.dropout1(self.fc1(x)))
        x = self.fc2(x)
        return x

## Обучение модели

In [441]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [442]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

In [443]:
!nvidia-smi

Sat Nov 20 15:50:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.00    Driver Version: 470.82.00    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   40C    P8     9W / 170W |   2279MiB / 12045MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [563]:
learning_rate = 0.0005
num_epochs = 30
margin = 0.1
batch_size = 32

In [564]:
faces_directory_path = '/home/dima/datasets/vgg_face_dataset/faces'

In [565]:
dataset = get_data_csv(faces_directory_path)
dataset.shape

(15228, 2)

In [566]:
train_df, valid_df, test_df = get_train_valid_test(source_df=dataset, separate_feature='person_name')
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

In [567]:
print(f"Unique person names in train: {train_df['person_name'].nunique()}")
print(f"Unique person names in valid: {valid_df['person_name'].nunique()}")
print(f"Unique person names in test: {test_df['person_name'].nunique()}")
print(f"Total images in train: {train_df.shape[0]}")
print(f"Total images in valid: {valid_df.shape[0]}")
print(f"Total images in test: {test_df.shape[0]}")

Unique person names in train: 120
Unique person names in valid: 40
Unique person names in test: 40
Total images in train: 9014
Total images in valid: 3146
Total images in test: 3068


In [568]:
research_data = ResearchDataset(train_df, device)
research_data_loader = DataLoader(research_data, batch_size=64, shuffle=True)
mean, std = get_mean_std(research_data_loader)

In [569]:
print(f'Mean channel values in train: {mean}')
print(f'Std channel values in train: {std}')

Mean channel values in train: (0.654159, 0.49055982, 0.41564453)
Std channel values in train: (0.20767309, 0.1787051, 0.16775697)


In [627]:
# Попробовать обучить модель со стандартизацией и без и сравнить качество
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=mean, std=std, max_pixel_value=1.0),
    ToTensorV2(),
])

valid_test_transform = A.Compose([
    A.Normalize(mean=mean, std=std, max_pixel_value=1.0),
    ToTensorV2(),
])

In [628]:
# Сравнить производительность при разных num_workers
train_data = CustomDataset(train_df, device, train_transform)
valid_data = CustomDataset(valid_df, device, valid_test_transform)
test_data = CustomDataset(test_df, device, valid_test_transform)

train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_data_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [629]:
# model from scratch
#model = Model().to(device)

# transfer learning with mobilenet v3 (imagenet)
#model = MobileImagenet().to(device)


network = NeuralNetwork(model=model)
summary(network, input_size=(32, 3, 160, 160))

Layer (type:depth-idx)                                  Output Shape              Param #
NeuralNetwork                                           --                        --
├─MobileImagenet: 1-1                                   [32, 128]                 --
│    └─MobileNetV3: 2-1                                 [32, 1024]                --
│    │    └─Sequential: 3-1                             [32, 576, 5, 5]           (927,008)
│    │    └─AdaptiveAvgPool2d: 3-2                      [32, 576, 1, 1]           --
│    │    └─Sequential: 3-3                             [32, 1024]                (590,848)
│    └─Linear: 2-2                                      [32, 256]                 262,400
│    └─Dropout: 2-3                                     [32, 256]                 --
│    └─Linear: 2-4                                      [32, 128]                 32,896
Total params: 1,813,152
Trainable params: 295,296
Non-trainable params: 1,517,856
Total mult-adds (M): 932.30
Input size (

In [630]:
#criterion = TripletLoss(margin=margin)
#metric = TripletLoss(margin=margin)
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
metric = nn.TripletMarginLoss(margin=1.0, p=2)

In [631]:
result_train = network.fit(criterion,
             metric,
             optimizer,
             train_data_loader,
             valid_data_loader,
             epochs=num_epochs)

Train Epoch: 1, Loss: 0.830728
Spended time for 50 batches (1600 triplets or 4800 images) : 5.329509 sec
Train Epoch: 1, Loss: 0.621594
Spended time for 50 batches (1600 triplets or 4800 images) : 5.395612 sec
Train Epoch: 1, Loss: 0.552309
Spended time for 50 batches (1600 triplets or 4800 images) : 5.382660 sec
Train Epoch: 1, Loss: 0.541826
Spended time for 50 batches (1600 triplets or 4800 images) : 5.415146 sec
Train Epoch: 1, Loss: 0.473931
Spended time for 50 batches (1600 triplets or 4800 images) : 5.341105 sec
Epoch 1, train loss: 0.588289, valid_loss: 0.576978, valid_metric: 0.576978
Train Epoch: 2, Loss: 0.458152
Spended time for 50 batches (1600 triplets or 4800 images) : 5.367517 sec
Train Epoch: 2, Loss: 0.444278
Spended time for 50 batches (1600 triplets or 4800 images) : 5.369329 sec
Train Epoch: 2, Loss: 0.413906
Spended time for 50 batches (1600 triplets or 4800 images) : 5.255110 sec
Train Epoch: 2, Loss: 0.437775
Spended time for 50 batches (1600 triplets or 4800 im

KeyboardInterrupt: 

In [ ]:
result_train

In [535]:
result_valid = network.valid(criterion, metric, valid_data_loader)
result_valid

{'valid_loss': 0.28777003751108143, 'valid_metric': 0.28777003751108143}

In [622]:
result_test = network.valid(criterion, metric, test_data_loader)
result_test

{'valid_loss': 0.4207154152294, 'valid_metric': 0.4207154152294}

In [536]:
network.save(path_to_save='./my_model_21ep_0.3test')

In [550]:
network = NeuralNetwork(model=model)
network.load(path_to_model = './my_model_21ep_0.3test')